# Ejercicio Extraccion de parrafos de un libro 

### Importacion de librerias

In [18]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, split, col, regexp_replace, size
import requests
import re

### Conexion con el cluster de spark y configuracion de instancia de sparkSession , asignamos el nombre "ExtracionLibro"  y verifica si la instancia ya fue creada o la crea

In [19]:
spark = SparkSession.builder.appName("ExtraccionLibro").getOrCreate()

### Url del libro en txt  libro : The Enchiridion by epicteto

In [20]:
url = "https://www.gutenberg.org/cache/epub/45109/pg45109.txt"

In [21]:
response = requests.get(url)
libro_texto =response.text

### guradamos la respuesta en un archivo de tipo txt

In [22]:
with open("TheEnchiridion.txt", "w", encoding = "utf-8") as file:
    file.write(libro_texto)

### Spark leera el archivo y lo convertira en u un RDD  


-RDD (Resilient Distributed Datasets =   son conjuntos de datos distribuidos  que pueden ser procesados de manera paralela en un cluster de Spark 
-Son colecciones distribuidas e inmutables de objetos de cualquier tipo  -
Son registros resilientes (tolerantes a fallas) de datos no estructurado  

Por defecto crea un elemento en el RDD cada que encuentra un salto de linea \n
 y lo guarda en la variable rdds

In [23]:
rdd = spark.sparkContext.textFile("TheEnchiridion.txt")

### Filtrado de elementos del RDD
aplicamos el metodo filter a la variable rdd, este metodo filtra por condicion  
aplicamos una funcion anonima que elimina los espacios en blanco ysaltos de linea  
Si la linea resultante es diferente de una cadana vacia entonces la guardamos

In [24]:
 rdd = rdd.filter(lambda line: line.strip() !="")

 creamos una expresion regular para que coincida con el inicio de los capitulos que en este libro son con numeros romanos 
 y creamos una lista que guarda las lineas que coinciden con el inicio de un capitulo 

In [25]:
numeros_romanos = r"^(L?X{0,3})(IX|IV|V?I{0,3})\b"
capitulos = rdd.filter(lambda line: re.match(numeros_romanos, line.strip())).collect()


aplicamos un metodo flatmap que aplica una funcion a cada elemento del RDD y al junta los resultados en un solo elemento, la funcion  sepára cada elemeto en donde exista un salto de linea \n\n y con la  funcion falt map lo volvemos a juntar para que se cuente como un solo parrafo

In [40]:

parrafos = rdd.flatMap(lambda line: line.split("\n\n"))

almacenamos en "df_parrafos un dataFrame  que con una columna llamada "parrafo"   
 con la lista de parrafos aplicamos el metod map  que aplica una funcion a cada elemento , le aplicacmos una funcion anonuma que por cada linea  cree una tupla de un solo elemento y la alacene  la columna del dataframe

In [27]:
df_parrafos = parrafos.map(lambda p: (p,)).toDF(["parrafo"])

In [41]:
# Mostramos los resultados
df_parrafos.show(50)

+--------------------+--------------------+--------------------+-------------+--------------------+------------+
|             parrafo|      parrafo_limpio|           oraciones|num_oraciones|            palabras|num_palabras|
+--------------------+--------------------+--------------------+-------------+--------------------+------------+
|The Project Guten...|The Project Guten...|[The Project Gute...|            1|[The, Project, Gu...|           7|
|This ebook is for...|This ebook is for...|[This ebook is fo...|            1|[This, ebook, is,...|          14|
|most other parts ...|most other parts ...|[most other parts...|            1|[most, other, par...|          14|
|whatsoever. You m...|whatsoever You ma...|[whatsoever You m...|            1|[whatsoever, You,...|          14|
|of the Project Gu...|of the Project Gu...|[of the Project G...|            1|[of, the, Project...|          11|
|at www.gutenberg....|at wwwgutenbergor...|[at wwwgutenbergo...|            1|[at, wwwgutenberg.

 agregamso una columna con los datos de la columna anterior pero aplicamos una  rexpresion regular para indicarle que elimine los caracteres que no conincidan con letras mayusculas o minusculas y espacion en blanco \s 

In [32]:
df_parrafos = df_parrafos.withColumn("parrafo_limpio", regexp_replace(col("parrafo"), "[^a-zA-Z\s]", "").alias("parrafo_limpio"))

reemplazamos multiple espacion en blanco por un solo espacio en blanco

In [33]:
df_parrafos = df_parrafos.withColumn("parrafo_limpio", regexp_replace(col("parrafo_limpio"), "\s+", " ").alias("parrafo_limpio"))

In [42]:
df_parrafos.show()

+--------------------+--------------------+--------------------+-------------+--------------------+------------+
|             parrafo|      parrafo_limpio|           oraciones|num_oraciones|            palabras|num_palabras|
+--------------------+--------------------+--------------------+-------------+--------------------+------------+
|The Project Guten...|The Project Guten...|[The Project Gute...|            1|[The, Project, Gu...|           7|
|This ebook is for...|This ebook is for...|[This ebook is fo...|            1|[This, ebook, is,...|          14|
|most other parts ...|most other parts ...|[most other parts...|            1|[most, other, par...|          14|
|whatsoever. You m...|whatsoever You ma...|[whatsoever You m...|            1|[whatsoever, You,...|          14|
|of the Project Gu...|of the Project Gu...|[of the Project G...|            1|[of, the, Project...|          11|
|at www.gutenberg....|at wwwgutenbergor...|[at wwwgutenbergo...|            1|[at, wwwgutenberg.

In [37]:
#agregamos un columna oraciones donde dividimos   cuando encuentre un  punto
df_parrafos = df_parrafos.withColumn("oraciones", split(col("parrafo_limpio"), "\."))
# agregamos una columan num_oraciones donde nos muestra el numeros de elementos mediante size
df_parrafos = df_parrafos.withColumn("num_oraciones", size(col("oraciones")))
#agregamso la columna palabras  diviendo la oracion por espacios
df_parrafos = df_parrafos.withColumn("palabras", split(col("parrafo_limpio"), " "))
#agregamos la columna  num_palabras contrando los elemento en pakabras mediante sizze
df_parrafos = df_parrafos.withColumn("num_palabras", size(col("palabras")))

### Resultados finales 

In [43]:
df_parrafos.show(60)

+--------------------+--------------------+--------------------+-------------+--------------------+------------+
|             parrafo|      parrafo_limpio|           oraciones|num_oraciones|            palabras|num_palabras|
+--------------------+--------------------+--------------------+-------------+--------------------+------------+
|The Project Guten...|The Project Guten...|[The Project Gute...|            1|[The, Project, Gu...|           7|
|This ebook is for...|This ebook is for...|[This ebook is fo...|            1|[This, ebook, is,...|          14|
|most other parts ...|most other parts ...|[most other parts...|            1|[most, other, par...|          14|
|whatsoever. You m...|whatsoever You ma...|[whatsoever You m...|            1|[whatsoever, You,...|          14|
|of the Project Gu...|of the Project Gu...|[of the Project G...|            1|[of, the, Project...|          11|
|at www.gutenberg....|at wwwgutenbergor...|[at wwwgutenbergo...|            1|[at, wwwgutenberg.